In [ ]:
%matplotlib inline
import os
os.environ['DATA_PATH'] = "/root/work/data"
import matplotlib.pyplot as plt
from invoke import task
from collections import OrderedDict
import numpy as np
from model import build_dense, build_conv, build_dense_resid, build_ciresan
import theano
import theano.tensor as T
from lasagne import layers, objectives, updates
from lasagnekit.datasets.mnist import MNIST
from lasagnekit.datasets.infinite_image_dataset import Transform
from helpers import iterate_minibatches, flip, rotate_scale, rotate_scale_one, elastic_transform, elastic_transform_one
from tabulate import tabulate
from time import time
from lasagnekit.misc.plot_weights import tile_raster_images

In [ ]:
print('Loading data...')
w, h, c = 28, 28, 1
def preprocess(data):
    return data.reshape((data.shape[0], c, w, h))

train = MNIST(which='train')
train.load()
train.X = preprocess(train.X)
#train.X = train.X[0:128]
#train.y = train.y[0:128]

test = MNIST(which='test')
test.load()
test.X = preprocess(test.X)

In [ ]:
%%time
#x = train.X[10].reshape((1, 28, 28))
#x = np.repeat(x, 100, axis=0)
x = train.X[:, 0]
x_before = x.copy()

x = rotate_scale(
    x,
    min_angle=-15, max_angle=15,
    min_scale=0.85, max_scale=1.15,
    n_jobs=-1)
x = elastic_transform( x, min_alpha=36, max_alpha=38, min_sigma=4, max_sigma=6, n_jobs=-1)

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.imshow(tile_raster_images(x_before, (28, 28), (10, 10)), cmap='gray', interpolation='none')
plt.subplot(1, 2, 2)
plt.imshow(tile_raster_images(x, (28, 28), (10, 10)), cmap='gray', interpolation='none')

In [ ]:
plt.hist(x_before.flatten(), normed=True, alpha=0.9)
plt.hist(x.flatten(), normed=True, alpha=0.9)